# Test whether: people who create manual entries have a more even distributed work time during the week, i.e., they work in a regular pace every day, registering the same average amount of hours per day.

## Read database and clean it up

In [35]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math

In [36]:
#read database
sheet_id = '1VK2ZQyVCNNjh4D-aQ4xyRjU8DrsLhQ9O02FnZ9S_nSU'
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

In [37]:
#clean database
#nan values
df = df.dropna(how="any", axis=0) 
#values equal to 0
df = df[(df != 0).all(1)]

In [38]:
#create new df with important columns 
dftask4 = df[['user_id', 'brokenruledescription', 'hours', 'spent_date']].copy()

## Classify users "The time entry is a manual entry running"

In [39]:
#filter the users with at least one brokenruledescription = The time entry is a manual entry running
dftask4 = dftask4.groupby(['spent_date','user_id','brokenruledescription']).agg({'hours':'sum'}).reset_index()

In [40]:
# Whats the name entry for people who create manual entries
dftask4.brokenruledescription.unique()

array(['Time entry bigger than 3 hours.',
       'The time entry is a manual entry stopped',
       'Time entry without any notes.'], dtype=object)

In [41]:
#classify entry type. Type 1 is for those who entered one or more "The time entry is a manual entry stopped", else, type -1.
dftask4['user_type'] = [1 if d == 'The time entry is a manual entry stopped' else -1 for d in dftask4['brokenruledescription']] 

In [42]:
#classify users by entrys classification
user_temp = ''

for i in dftask4.index: 
    if(dftask4["user_id"][i] == user_temp):
        tipoUser = dftask4["user_type"][i]
        dftask4["user_type"][i] = tipoUserTemp

    user_temp = dftask4["user_id"][i]
    tipoUserTemp = dftask4["user_type"][i]
    

<ipython-input-42-6f95ea30f3e1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftask4["user_type"][i] = tipoUserTemp


## Create DF for users type 1

In [43]:
#create users1 df
dfusers14 = dftask4[dftask4['user_type'] == 1]

In [44]:
#group by day
dfusers14 = dfusers14.groupby(['spent_date','user_id']).agg({'hours':'sum'}).reset_index()

## Create DF for all users

In [45]:
dftask4all = dftask4.groupby(['spent_date','user_id']).agg({'hours':'sum'}).reset_index()

## Run t test

In [46]:
#Run the T-test for the mean of ONE group of scores. See at: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html
stats.ttest_1samp(a = dfusers14['hours'], popmean = dftask4all['hours'].mean())

Ttest_1sampResult(statistic=-6.103810640967823, pvalue=3.5107658841576125e-08)

## Run correlation test

In [47]:
dftask4['user_type'].corr(dftask4['hours'])

-0.05853687472846683